# The Drazin Inverse and it's applications

### Testing for the conditions of a Drazin inverse

In [19]:
def drazin_tester(A, AD, k: int):
    
    import numpy as np
    
    A = np.array(A)
    AD = np.array(AD) 
    
    if np.allclose(np.matmul(A, AD), np.matmul(AD, A)) and \
     np.allclose(np.matmul(np.linalg.matrix_power(A, k+1), AD), np.linalg.matrix_power(A, k)) and \
     np.allclose(np.matmul(AD ,np.matmul(A, AD)), AD):
        return True
    else:
        return False

In [20]:
#Test 1
A = [[1, 3, 0, 0], [0, 1, 3, 0], [0, 0, 1, 3], [0, 0, 0, 0]]
AD = [[1, -3, 9, 81], [0, 1, -3, -18], [0, 0, 1, 3], [0, 0, 0, 0]]
k = 1

In [21]:
drazin_tester(A, AD, k) #OK!

True

In [22]:
# Test 2
import numpy as np
A = [[1, 1, 3], [5, 2, 6], [-2, -1, -3]]
AD = np.zeros([3, 3])
k = 3

In [23]:
drazin_tester(A, AD, k) #OK!

True

### Calculating the Drazin inverse of matrices

In [24]:
from scipy import linalg as la
import numpy as np
def drazin(A, tol):
    A = np.array(A)
    n, n = A.shape
    Q1, S, k1 = la.schur(A, sort = lambda x: abs(x) > tol)
    Q2, T, k2 = la.schur(A, sort = lambda x: abs(x) <= tol)
    U = np.concatenate((S[:, :k1], T[:, :n-k1]), axis = -1)
    Uinv = np.linalg.inv(U)
    V = np.matmul(Uinv, np.matmul(A, U))
    Z = np.zeros([n, n])
    if k1 != 0:
        Minv = np.linalg.inv(V[:k1, :k1])
        Z[:k1, :k1] = Minv
    return np.matmul(U, np.matmul(Z, Uinv))

In [25]:
A = [[1, 3, 0, 0], [0, 1, 3, 0], [0, 0, 1, 3], [0, 0, 0, 0]]
drazin(A, 0.005)

array([[  1.,  -3.,   9.,  81.],
       [  0.,   1.,  -3., -18.],
       [  0.,   0.,   1.,   3.],
       [  0.,   0.,   0.,   0.]])

In [26]:
A = [[1, 1, 3], [5, 2, 6], [-2, -1, -3]]
drazin(A, 0.005)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

### Effective Resistence

In [27]:
import networkx as nx

def drazin_resistance(A):
    
    A = np.array(A)
    n, n = A.shape
    I = np.eye(n)
    
    # Calculate resistence for each node
    R = np.zeros_like(A, dtype = float)
    for i in range(n):
        for j in range(n):
            if i != j:
                G = nx.from_numpy_array(A)
                L = nx.laplacian_matrix(G).toarray()
                L[:, j] = I[:, j]
                R[i, j] = drazin(L, 0.05)[i, i]
            else:
                pass
    
    return R

In [28]:
# Connected line with 4 nodes
A = [[0, 1, 0, 0], [1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 0, 1]]
drazin_resistance(A)

array([[0., 1., 2., 3.],
       [1., 0., 1., 2.],
       [2., 1., 0., 1.],
       [3., 2., 1., 0.]])

In [29]:
# Connected line with 2 nodes
A = [[0, 1], [1, 0]]
drazin_resistance(A)

array([[0., 1.],
       [1., 0.]])

In [30]:
# Connected triangle
A = [[0, 1, 1], [1, 0, 1], [0, 1, 1]]
drazin_resistance(A)

array([[0.        , 0.66666667, 0.66666667],
       [0.66666667, 0.        , 0.66666667],
       [0.66666667, 0.66666667, 0.        ]])

In [31]:
# Circle with one connection in the middle
A = [[0, 3], [3, 0]]
drazin_resistance(A)

array([[0.        , 0.33333333],
       [0.33333333, 0.        ]])

In [32]:
# Circle
A = [[0, 2], [2, 0]]
drazin_resistance(A)

array([[0. , 0.5],
       [0.5, 0. ]])

In [33]:
# Circle with two connection in the middle
A = [[0, 4], [4, 0]]
drazin_resistance(A)

array([[0.  , 0.25],
       [0.25, 0.  ]])

### Link prediction (finding the least resistence)

In [34]:
class LinkPredictor:
    def __init__(self, file):
        
        import pandas as pd
        nw = pd.read_csv(file, header = None)
        
        # List of names
        names = sorted(list(dict.fromkeys(list(nw[0].unique()) + list(nw[1].unique()))))
        
        # Creating adjency matrix
        network1 = []
        for i in names:
            row = [0]*len(names)
            for j in range(len(nw)):
                if i == nw.iloc[j, 0]:
                    row[names.index(nw.iloc[j, 1])] = 1
                else:
                    pass
            network1 += ((row), )
        
        network2 = []
        for k in names:
            row = [0]*len(names)
            for l in range(len(nw)):
                if k == nw.iloc[l, 1]:
                    row[names.index(nw.iloc[l, 0])] = 1
                else:
                    pass
            network2 += ((row), )
        
        network = np.array(network1) + np.array(network2)
        adj_mat = np.array(network)
        
        # Creating the effective resistence matrix
        res_mat = drazin_resistance(adj_mat)
        
        self.names = names
        self.adj_mat = adj_mat
        self.res_mat = res_mat

In [35]:
social_network = LinkPredictor('social_network.csv') #From https://en.wikipedia.org/wiki/Zachary%27s_karate_club

In [36]:
social_network.names

['Abigail',
 'Alan',
 'Alexander',
 'Anna',
 'Brandon',
 'Carol',
 'Charles',
 'Christopher',
 'Colin',
 'Connor',
 'Emily',
 'Eric',
 'Evan',
 'Jake',
 'Jane',
 'John',
 'Madeleine',
 'Mary',
 'Max',
 'Melanie',
 'Oliver',
 'Paul',
 'Penelope',
 'Piers',
 'Ruth',
 'Sally',
 'Sonia',
 'Stephanie',
 'Stephen',
 'Theresa',
 'Thomas',
 'Tracey',
 'Trevor']

In [37]:
social_network.adj_mat

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [38]:
social_network.res_mat

array([[0.        , 0.70030688, 0.76301021, ..., 0.4795326 , 0.39108078,
        0.65489609],
       [0.70030688, 0.        , 1.06076045, ..., 0.80050616, 0.74156341,
        1.12822586],
       [0.76301021, 1.06076045, 0.        , ..., 0.82885358, 0.74028882,
        1.18497957],
       ...,
       [0.4795326 , 0.80050616, 0.82885358, ..., 0.        , 0.45481592,
        0.88646385],
       [0.39108078, 0.74156341, 0.74028882, ..., 0.45481592, 0.        ,
        0.77032698],
       [0.65489609, 1.12822586, 1.18497957, ..., 0.88646385, 0.77032698,
        0.        ]])

### Problem 6

In [39]:
class LinkPredictor:
    def __init__(self, file):
        
        import pandas as pd
        nw = pd.read_csv(file, header = None)
        
        # List of names
        names = sorted(list(dict.fromkeys(list(nw[0].unique()) + list(nw[1].unique()))))
        
        # Creating adjency matrix
        network1 = []
        for i in names:
            row = [0]*len(names)
            for j in range(len(nw)):
                if i == nw.iloc[j, 0]:
                    row[names.index(nw.iloc[j, 1])] = 1
                else:
                    pass
            network1 += ((row), )
        
        network2 = []
        for k in names:
            row = [0]*len(names)
            for l in range(len(nw)):
                if k == nw.iloc[l, 1]:
                    row[names.index(nw.iloc[l, 0])] = 1
                else:
                    pass
            network2 += ((row), )
        
        network = np.array(network1) + np.array(network2)
        adj_mat = np.array(network)
        
        # Creating the effective resistence matrix
        res_mat = drazin_resistance(adj_mat)
        
        self.names = names
        self.adj_mat = adj_mat
        self.res_mat = res_mat
        
    def predict_link(self, node = None):
        
        if node is not None and node in self.names:
            An = -(self.adj_mat - 1)
            Mpot = An*self.res_mat
            Mpot[Mpot == 0] = np.inf
            index = self.names.index(node)
            min_val = np.min(Mpot[:, index])
            coords = np.where(Mpot==min_val)
            return (self.names[int(coords[0])], self.names[int(coords[1])])
            
        elif node is None:
            An = -(self.adj_mat - 1)
            Mpot = An*self.res_mat
            Mpot[Mpot == 0] = np.inf
            min_val = np.min(Mpot)
            coords = np.where(Mpot==min_val)
            return (self.names[int(coords[0])], self.names[int(coords[1])])
        else:
            raise ValueError("The specified node is not in the network!")
            
    def add_link(self, name1: str, name2: str):
        
        if name1 not in self.names or name2 not in self.names:
            raise ValueError('Please specify names that can already be found in the network!')
        
        if name1 == name2:
            raise ValueError('Connection cannot exist between the person and himself/herself.')
            
        if self.adj_mat[self.names.index(name1), self.names.index(name2)] == 0:
            self.adj_mat[self.names.index(name1), self.names.index(name2)] = 1
            self.adj_mat[self.names.index(name2), self.names.index(name1)] = 1
            self.res_mat = drazin_resistance(self.adj_mat)
        else:
            raise ValueError('The connection between these nodes is already exists!')

In [40]:
social_network = LinkPredictor('social_network.csv')

In [41]:
social_network.predict_link() # I think it is OK!

('Emily', 'Connor')

In [42]:
social_network.predict_link('Melanie') #OK!

('Carol', 'Melanie')

In [43]:
social_network.predict_link('Alan') #OK!

('Sonia', 'Alan')

In [44]:
social_network.add_link('Abigail', 'Alan')

In [45]:
social_network.add_link('Alan', 'Alan') # It is good! Someone cannot be linket to him/herself!

ValueError: Connection cannot exist between the person and himself/herself.